In [1]:
#Import de librerías necesarias
import pyreadr
import matplotlib.pyplot as plt
from matplotlib.widgets import Cursor
import numpy as np
import pandas as pd
from datetime import datetime, timedelta, date
from sklearn import *
from sklearn.model_selection import *
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import random
import time
import pandas as pd
from scipy.stats import chi2
from scipy.stats import chi2_contingency
import scipy.stats as stats
from sklearn.metrics import *
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report
from scipy.stats import mannwhitneyu
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from collections import defaultdict
from statsmodels.stats.outliers_influence import variance_inflation_factor 
from scipy.stats import spearmanr, kendalltau
from sklearn.linear_model import LassoCV
from sklearn.linear_model import ElasticNetCV
from sklearn import linear_model
from boruta import BorutaPy
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeClassifier
from mlxtend.feature_selection import SequentialFeatureSelector
from sklearn.utils.class_weight import compute_class_weight
from patsy import cr
from sklearn.tree import DecisionTreeRegressor
import xgboost as xgb
import cupy as cp
from sklearn.model_selection import RandomizedSearchCV

In [2]:
np.int = np.int32
np.float = np.float64
np.bool = np.bool_

In [3]:
#Importar los datos de R a Python
datos = pyreadr.read_r('area_desbalance_ionico.RData')

#Para comprobar que se importan correctamente los datos
#print(datos.keys())

#Este archivo tiene tres variables

#datos_anon_D: datos extraídos de los ECGS - edad, sexo, fecha, hora y 
#nombre del fichero. Hay un código (código) que identifica a cada paciente
datos_anon_DF = datos['datos_anon_DF']

#lead_anon_DF
#variables electrocardiográficas correspondiente a los ECGs de la variable anterior
leads_anon_DF = datos['leads_anon_DF']

#pacs_con_ECGs_DF
#variables de los datos del potasio, fecha de la analítica, edad del paciente, sexo
#nivel de K y código del paciente
pacs_con_ECGs_DF = datos['pacs_con_ECGs_DF']

# Normalidad - Hiperpotasemia - Hipopotasemia

In [4]:
#Primer paso: coger solo los valores de K normales y de hiperpotasemia
k_val = pacs_con_ECGs_DF.copy()

limites = [float('-inf'), 3.5, 5.2, float('inf')]
etiquetas = [1, 0, 2]

In [5]:
k_valores = k_val.copy()

k_valores['categoria'] = pd.cut(k_valores['K'], bins=limites, labels=etiquetas, right=False)

#Se reemplazan los sexos por enteros
#1: Hombre
#0: Mujer
k_valores['Sexo'] = k_valores['Sexo'].replace({'M': 1, 'F': 0})

# Eliminación de variables con datos vacíos

In [6]:
#En leads_anon_DF, se van a eliminar aquellas columnas en las que falte más del 1% de los datos

#El como máximo (el 100%) de datos que podría tener cada característica, sería igual al número total de filas que
#hay registradas
#Se calcula el 10% y se redondea sin decimales
diez_pct_datos = round(len(leads_anon_DF) * 0.01, 0)

#Ahora, se va a coger un subconjunto del dataframe en el que se eliminan las
#características cuyo número de datos sea menor al 10%

#Se cuenta el número de NaN o celdas sin datos en cada columna
celdas_sin_datos = leads_anon_DF.isnull().sum()

#Se cogen las características cuyo número de datos vacíos es mayor o igual al 10%
caracteristicas_eliminar = celdas_sin_datos[celdas_sin_datos >= diez_pct_datos].index

#Se crear el subDataFrame eliminando las columnas correspondientes
leads_anon_DF_limpio = leads_anon_DF.drop(columns = caracteristicas_eliminar)

In [7]:
#Ahora hay que asociar a cada análisis de sangre, su ECG más cercano
def ECG_mas_reciente(fecha, codigo):
     
    archivos_ECGs = datos_anon_DF.loc[(datos_anon_DF['codigo'] == codigo) & (datos_anon_DF['date'] >= (fecha - timedelta(days=5))) & (datos_anon_DF['date'] <= (fecha + timedelta(days=5)))]    
    
    fechas_archivos = archivos_ECGs['date']
    horas_archivos = archivos_ECGs['time']
        
    diferencia_temporal = [abs(fecha - fecha_archivo) for fecha_archivo in fechas_archivos]
    
    ecg = (datos_anon_DF.index[(datos_anon_DF['codigo'] == codigo) & (datos_anon_DF['date'] >= (fecha - timedelta(days=5))) & (datos_anon_DF['date'] <= (fecha + timedelta(days=5)))]).tolist()
    
    if(len(diferencia_temporal) > 0):
    
        combinado = list(zip(diferencia_temporal, ecg))
        combinado = sorted(combinado, key=lambda x: x[0])

        diferencia_temporal, ecg = zip(*combinado)
        
    return ecg, diferencia_temporal

In [8]:
#Ahora, se añade la columna "Categoria" y "Sexo" a leads_anon_DF_limpio
#Se crean esas columnas en el nuevo dataframe con valores a Nan
leads_anon_DF_limpio['Sexo'] = float('NaN')
leads_anon_DF_limpio['categoria'] = float('NaN')
leads_anon_DF_limpio['K'] = float('NaN')
leads_anon_DF_limpio['Edad'] = float('NaN')
ecgs_utilizadas = {} 

for i in range(len(k_valores)):
    fecha_hora = (k_valores['Fecha'].iloc[i]).to_pydatetime()
    fecha = fecha_hora.date()
    
    indice, dist_temporal = ECG_mas_reciente(fecha, k_valores['codigo'].iloc[i])
  
    for j in range(len(indice)):
        #Si el ECGs no tiene una analítica asignada, se le pone
        if indice[j] not in ecgs_utilizadas:
            #Se actualizan los valores de las columnas "Sexo" y "Categoría"
            #de la fila correspondiente
            leads_anon_DF_limpio.at[indice[j], 'Sexo'] = k_valores['Sexo'].iloc[i]
            leads_anon_DF_limpio.at[indice[j], 'Edad'] = k_valores['Edad'].iloc[i]
            leads_anon_DF_limpio.at[indice[j], 'categoria'] = k_valores['categoria'].iloc[i]
            leads_anon_DF_limpio.at[indice[j], 'K'] = k_valores['K'].iloc[i]

            ecgs_utilizadas[indice[j]] = [k_valores['categoria'].iloc[i], dist_temporal[j].days]

        #Si el ECG ya tiene una analítica asociada, si la nueva analítica tiene un valor
        #de categoría diferente y se asocia con una anormalidad, se cambia
        else:
            if((dist_temporal[j].days < ecgs_utilizadas[indice[j]][1])):
                leads_anon_DF_limpio.at[indice[j], 'categoria'] = k_valores['categoria'].iloc[i]
                leads_anon_DF_limpio.at[indice[j], 'K'] = k_valores['K'].iloc[i]
                ecgs_utilizadas[indice[j]][1] = dist_temporal[j].days
                
            if((dist_temporal[j].days == ecgs_utilizadas[indice[j]][1]) & (k_valores['categoria'].iloc[i] > ecgs_utilizadas[indice[j]][0])):
                leads_anon_DF_limpio.at[indice[j], 'categoria'] = k_valores['categoria'].iloc[i]
                leads_anon_DF_limpio.at[indice[j], 'K'] = k_valores['K'].iloc[i]
                ecgs_utilizadas[indice[j]][1] = dist_temporal[j].days                
            
#Se eliminan las filas que tengan la variable objetivo "categoria" a NaN
leads_anon_DF_limpio = leads_anon_DF_limpio.dropna(subset=['categoria'])

In [9]:
pparea = leads_anon_DF_limpio.filter(regex='_pparea', axis=1)
tparea = leads_anon_DF_limpio.filter(regex='_tparea', axis=1)

ppamp = leads_anon_DF_limpio.filter(regex='_ppamp', axis=1)
rpamp = leads_anon_DF_limpio.filter(regex='_rpamp', axis=1)
spamp = leads_anon_DF_limpio.filter(regex='_spamp', axis=1)
tpamp = leads_anon_DF_limpio.filter(regex='_tpamp', axis=1)

rpdur = leads_anon_DF_limpio.filter(regex='_rpdur', axis=1)
spdur = leads_anon_DF_limpio.filter(regex='_spdur', axis=1)
tpdur = leads_anon_DF_limpio.filter(regex='_tpdur', axis=1)

leads_anon_DF_limpio = leads_anon_DF_limpio.drop(columns=pparea.columns)
leads_anon_DF_limpio = leads_anon_DF_limpio.drop(columns=tparea.columns)
leads_anon_DF_limpio = leads_anon_DF_limpio.drop(columns=ppamp.columns)
leads_anon_DF_limpio = leads_anon_DF_limpio.drop(columns=rpamp.columns)
leads_anon_DF_limpio = leads_anon_DF_limpio.drop(columns=spamp.columns)
leads_anon_DF_limpio = leads_anon_DF_limpio.drop(columns=tpamp.columns)
leads_anon_DF_limpio = leads_anon_DF_limpio.drop(columns=rpdur.columns)
leads_anon_DF_limpio = leads_anon_DF_limpio.drop(columns=spdur.columns)
leads_anon_DF_limpio = leads_anon_DF_limpio.drop(columns=tpdur.columns)

In [10]:
parea = leads_anon_DF_limpio.filter(regex='_parea', axis=1)
pppparea = leads_anon_DF_limpio.filter(regex='_pppparea', axis=1)
qdur = leads_anon_DF_limpio.filter(regex='_qdur', axis=1)
sdur = leads_anon_DF_limpio.filter(regex='_sdur', axis=1)
tarea = leads_anon_DF_limpio.filter(regex='_tarea', axis=1)
tptparea = leads_anon_DF_limpio.filter(regex='(_tptparea$)', axis=1)
tptpdur = leads_anon_DF_limpio.filter(regex='(_tptpdur$)', axis=1)
st = leads_anon_DF_limpio.filter(regex='(_stend$|_st80$|_ston$)', axis=1)
stslope = leads_anon_DF_limpio.filter(regex='(_stslope)', axis=1)
stdur = leads_anon_DF_limpio.filter(regex='(_stdur$)', axis=1).drop(columns=['V2_stdur'])

leads_anon_DF_limpio = leads_anon_DF_limpio.drop(columns=parea.columns)
leads_anon_DF_limpio = leads_anon_DF_limpio.drop(columns=pppparea.columns)
leads_anon_DF_limpio = leads_anon_DF_limpio.drop(columns=qdur.columns)
leads_anon_DF_limpio = leads_anon_DF_limpio.drop(columns=sdur.columns)
leads_anon_DF_limpio = leads_anon_DF_limpio.drop(columns=tarea.columns)
leads_anon_DF_limpio = leads_anon_DF_limpio.drop(columns=tptparea.columns)
leads_anon_DF_limpio = leads_anon_DF_limpio.drop(columns=tptpdur.columns)
leads_anon_DF_limpio = leads_anon_DF_limpio.drop(columns=st.columns)
leads_anon_DF_limpio = leads_anon_DF_limpio.drop(columns=stslope.columns)
leads_anon_DF_limpio = leads_anon_DF_limpio.drop(columns=stdur.columns)

# CARGA DE DATOS FINALIZADA

### Hipopotasemia

In [43]:
x = leads_anon_DF_limpio[leads_anon_DF_limpio['categoria'] != 2]
y = x['categoria'].astype(int)

x = x.drop(['K', 'categoria'], axis = 1)
x = x.where(pd.notna(x), None)

In [44]:
cars_aic = ['aVL_pamp', 'II_ramp', 'V5_rdur', 'aVL_vat', 'V3_vat', 'aVL_qrsppk', 'V1_qrsppk', 'III_qrsdur', 'V1_qrsdur', 'V3_stmid', 'V6_stmid', 'V2_stdur', 'aVR_tamp', 'V1_tamp', 'V5_tamp', 'I_tdur', 'V6_tdur']
cars_bor = ['aVL_pamp', 'V4_ramp', 'I_stmid', 'aVR_stmid', 'V2_stmid', 'V3_stmid', 'V2_stdur', 'aVR_tamp', 'V2_tamp', 'V3_tamp']

caracteristicas = [cars_aic, cars_bor]
modelos = ['AIC', 'Boruta']

In [ ]:
imputer = IterativeImputer(max_iter=5, sample_posterior=True, initial_strategy='mean', skip_complete=True)

x = pd.DataFrame(imputer.fit_transform(x), columns=x.columns)

esc = StandardScaler()
x = pd.DataFrame(esc.fit_transform(x), columns=x.columns)

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)

### Hiperpotasemia

In [50]:
x = leads_anon_DF_limpio[leads_anon_DF_limpio['categoria'] != 1]
y = x['categoria'].astype(int)
y = y.apply(lambda x: 1 if x == 2 else 0)

x = x.drop(['K', 'categoria'], axis = 1)
x = x.where(pd.notna(x), None)

In [51]:
cars_aic = ['V5_qamp', 'I_rdur', 'V5_samp', 'V5_vat', 'V1_qrsdur', 'aVL_qrsarea', 'V1_qrsarea', 'V3_qrsarea', 'V3_stmid', 'V2_tamp', 'V6_qtint']
cars_bor = ['I_ramp', 'V5_samp', 'I_qrsppk', 'II_qrsppk', 'aVR_qrsppk', 'V1_qrsppk', 'V3_qrsppk', 'V4_qrsppk', 'V5_qrsppk', 'V1_qrsdur', 'V5_qrsarea', 'V4_stmid', 'V2_tamp', 'V3_tamp', 'aVL_tdur', 'III_qtint', 'V6_qtint', 'Edad']
caracteristicas = [cars_aic, cars_bor]
modelos = ['AIC', 'Boruta']

In [ ]:
imputer = IterativeImputer(max_iter=5, sample_posterior=True, initial_strategy='mean', skip_complete=True)

x = pd.DataFrame(imputer.fit_transform(x), columns=x.columns)

esc = StandardScaler()
x = pd.DataFrame(esc.fit_transform(x), columns=x.columns)

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)

### Regresión

In [59]:
x = leads_anon_DF_limpio
x = x.sample(frac=1, random_state=1)
y = x['K']

x = x.drop(['K', 'categoria'], axis = 1)
x = x.where(pd.notna(x), None)

In [60]:
cars_aic = ['I_rdur', 'I_qrsarea', 'V1_qrsarea', 'V2_stdur', 'V2_tamp', 'V6_qtint']
cars_boruta = ['V1_pamp', 'II_ramp', 'V5_samp', 'III_vat', 'V1_vat', 'I_qrsppk', 'aVF_qrsppk', 'V1_qrsppk', 'V4_qrsppk', 'V2_qrsarea', 'I_stmid', 'V3_stmid', 'aVR_tamp', 'V1_tamp', 'V2_tamp', 'V3_tamp', 'I_tdur', 'aVL_tdur', 'V6_tdur', 'Edad']

caracteristicas = [cars_aic, cars_bor]
modelos = ['AIC', 'Boruta']

In [ ]:
imputer = IterativeImputer(max_iter=5, sample_posterior=True, initial_strategy='mean', skip_complete=True)

x = pd.DataFrame(imputer.fit_transform(x), columns=x.columns)

esc = StandardScaler()
x = pd.DataFrame(esc.fit_transform(x), columns=x.columns)

kf = KFold(n_splits=5, shuffle=True, random_state=1)

---

# CARTs

### Hipopotasemia

In [17]:
parametros = {
    'max_depth': [None, 10, 25, 50, 100],
    'max_features': [1, 5, 10, None], 
    'max_leaf_nodes': [None, 5, 10, 25, 50]
    
}

ad = DecisionTreeClassifier(class_weight='balanced')

for i in range(len(caracteristicas)):
    x2 = x[caracteristicas[i]]

    grid = GridSearchCV(ad, parametros, cv=skf, scoring='roc_auc', n_jobs=-1, verbose=2)

    grid.fit(x2, y)

    print(f' ')
    print(f"Mejores parámetros {modelos[i]}: {grid.best_params_}")

Fitting 5 folds for each of 100 candidates, totalling 500 fits
 
Mejores parámetros AIC: {'max_depth': 10, 'max_features': 10, 'max_leaf_nodes': 10}
Fitting 5 folds for each of 100 candidates, totalling 500 fits
 
Mejores parámetros Boruta: {'max_depth': None, 'max_features': 5, 'max_leaf_nodes': 10}


### Hiperpotasemia

In [31]:
parametros = {
    'max_depth': [None, 10, 25, 50, 100],
    'max_features': [1, 5, 10, None], 
    'max_leaf_nodes': [None, 5, 10, 25, 50]
    
}

ad = DecisionTreeClassifier(class_weight='balanced')

for i in range(len(caracteristicas)):
    x2 = x[caracteristicas[i]]

    grid = GridSearchCV(ad, parametros, cv=skf, scoring='roc_auc', n_jobs=-1, verbose=2)

    grid.fit(x2, y)

    print(f' ')
    print(f"Mejores parámetros {modelos[i]}: {grid.best_params_}")

Fitting 5 folds for each of 100 candidates, totalling 500 fits
 
Mejores parámetros AIC: {'max_depth': None, 'max_features': 10, 'max_leaf_nodes': 50}
Fitting 5 folds for each of 100 candidates, totalling 500 fits
 
Mejores parámetros Boruta: {'max_depth': 100, 'max_features': 10, 'max_leaf_nodes': 50}


### Regresión

In [40]:
parametros = {
    'max_depth': [None, 10, 25, 50, 100],
    'max_features': [1, 5, 10, None], 
    'max_leaf_nodes': [None, 5, 10, 25, 50]
    
}

ad = DecisionTreeRegressor(criterion='squared_error')

for i in range(len(caracteristicas)):
    x2 = x[caracteristicas[i]]

    grid = GridSearchCV(ad, parametros, cv=5, scoring='r2', n_jobs=-1)

    grid.fit(x2, y)

    print(f"Mejores parámetros {modelos[i]}: {grid.best_params_}")

Mejores parámetros AIC: {'max_depth': 100, 'max_features': 1, 'max_leaf_nodes': 5}
Mejores parámetros Boruta: {'max_depth': 100, 'max_features': 10, 'max_leaf_nodes': 5}


# Random Forest

### Hipopotasemia

In [88]:
parametros = {
    'n_estimators': [100, 150, 200, 250],
    'max_depth': [None, 10, 25, 50, 100],
    'max_features': [5, 10, None], 
    'max_leaf_nodes': [None, 5, 10, 25, 50]
}

ad = RandomForestClassifier(n_jobs=-1, oob_score=roc_auc_score, class_weight='balanced')

for i in range(len(caracteristicas)):
    x2 = x[caracteristicas[i]]

    grid = RandomizedSearchCV(ad, parametros, n_iter=100, cv=skf, scoring='roc_auc', n_jobs=-1)

    grid.fit(x2, y)

    print(f"Mejores parámetros {modelos[i]}: {grid.best_params_}")

Mejores parámetros AIC: {'n_estimators': 150, 'max_leaf_nodes': None, 'max_features': None, 'max_depth': 25}
Mejores parámetros Boruta: {'n_estimators': 150, 'max_leaf_nodes': None, 'max_features': None, 'max_depth': None}


### Hiperpotasemia

In [32]:
parametros = {
    'n_estimators': [100, 150, 200, 250],
    'max_depth': [None, 10, 25, 50, 100],
    'max_features': [5, 10, None], 
    'max_leaf_nodes': [None, 5, 10, 25, 50]
}

ad = RandomForestClassifier(n_jobs=-1, oob_score=roc_auc_score, class_weight='balanced')

for i in range(len(caracteristicas)):
    x2 = x[caracteristicas[i]]

    grid = RandomizedSearchCV(ad, parametros, n_iter=100, cv=skf, scoring='roc_auc', n_jobs=-1)

    grid.fit(x2, y)

    print(f"Mejores parámetros {modelos[i]}: {grid.best_params_}")

Mejores parámetros AIC: {'n_estimators': 200, 'max_leaf_nodes': None, 'max_features': None, 'max_depth': 100}
Mejores parámetros Boruta: {'n_estimators': 200, 'max_leaf_nodes': None, 'max_features': 5, 'max_depth': None}


### Regresión

In [41]:
parametros = {
    'n_estimators': [100, 150, 200, 250],
    'max_depth': [None, 10, 25, 50, 100],
    'max_features': [5, 10, None], 
    'max_leaf_nodes': [None, 5, 10, 25, 50]
}

ad = RandomForestRegressor(n_jobs=-1, oob_score=r2_score)

for i in range(len(caracteristicas)):
    x2 = x[caracteristicas[i]]

    grid = RandomizedSearchCV(ad, parametros, n_iter=100, scoring='r2', n_jobs=-1)

    grid.fit(x2, y)

    print(f"Mejores parámetros {modelos[i]}: {grid.best_params_}")

Mejores parámetros AIC: {'n_estimators': 250, 'max_leaf_nodes': None, 'max_features': 5, 'max_depth': None}
Mejores parámetros Boruta: {'n_estimators': 200, 'max_leaf_nodes': None, 'max_features': 10, 'max_depth': None}


# XGBoost

### Hipopotasemia

In [49]:
parametros = {
    'n_estimators': [100, 150, 200, 250],
    'max_depth': [0, 10, 25, 50, 100],
    'max_leaves': [0, 5, 10, 25, 50],
    'learning_rate': [0.01, 0.1, 0.25, 0.5]
}

pesos = (len(y[y == 0]) / len(y[y==1]))

ad = xgb.XGBClassifier(device="cuda", n_jobs=-1, eval_metric=roc_auc_score, scale_pos_weight=pesos)

for i in range(len(caracteristicas)):
    x2 = x[caracteristicas[i]]

    grid = RandomizedSearchCV(ad, parametros, n_iter=100, cv=skf, scoring='roc_auc', n_jobs=-1)

    grid.fit(cp.array(x2), y)

    print(f' ')
    print(f"Mejores parámetros {modelos[i]}: {grid.best_params_}")

 
Mejores parámetros AIC: {'n_estimators': 100, 'max_leaves': 50, 'max_depth': 50, 'learning_rate': 0.5}
 
Mejores parámetros Boruta: {'n_estimators': 250, 'max_leaves': 25, 'max_depth': 25, 'learning_rate': 0.1}


### Hiperpotasemia

In [56]:
parametros = {
    'n_estimators': [100, 150, 200, 250],
    'max_depth': [0, 10, 25, 50, 100],
    'max_leaves': [0, 5, 10, 25, 50],
    'learning_rate': [0.01, 0.1, 0.25, 0.5]
}

pesos = (len(y[y == 0]) / len(y[y==1]))

ad = xgb.XGBClassifier(device="cuda", n_jobs=-1, eval_metric=roc_auc_score, scale_pos_weight=pesos)

for i in range(len(caracteristicas)):
    x2 = x[caracteristicas[i]]

    grid = RandomizedSearchCV(ad, parametros, n_iter=100, cv=skf, scoring='roc_auc', n_jobs=-1)

    grid.fit(cp.array(x2), y)

    print(f' ')
    print(f"Mejores parámetros {modelos[i]}: {grid.best_params_}")

 
Mejores parámetros AIC: {'n_estimators': 250, 'max_leaves': 0, 'max_depth': 25, 'learning_rate': 0.1}
 
Mejores parámetros Boruta: {'n_estimators': 250, 'max_leaves': 0, 'max_depth': 50, 'learning_rate': 0.25}


### Regresión

In [65]:
parametros = {
    'n_estimators': [100, 150, 200, 250],
    'max_depth': [0, 10, 25, 50, 100],
    'max_leaves': [0, 5, 10, 25, 50],
    'learning_rate': [0.01, 0.1, 0.25, 0.5]
}

ad = xgb.XGBRegressor(device="cuda", n_jobs=-1, eval_metric=r2_score)

for i in range(len(caracteristicas)):
    x2 = x[caracteristicas[i]]

    grid = RandomizedSearchCV(ad, parametros, n_iter=100, scoring='r2', n_jobs=-1)

    grid.fit(cp.array(x2), y)

    print(f"Mejores parámetros {modelos[i]}: {grid.best_params_}")

Mejores parámetros AIC: {'n_estimators': 150, 'max_leaves': 10, 'max_depth': 25, 'learning_rate': 0.1}
Mejores parámetros Boruta: {'n_estimators': 200, 'max_leaves': 0, 'max_depth': 10, 'learning_rate': 0.1}
